# Running Tune experiments with AxSearch

This example demonstrates the usage of AxSearch with Ray Tune via `AxSearch`.

It also also shows that a scheduler can be used with `AxSearch`, e.g. `AsyncHyperBandScheduler`.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

Necessary requirements:
- `pip install ray[tune]`
- `pip install ax-platform==0.1.9` for python version < '3.7'
- `pip install ax-platform==0.2.4` for python version >= '3.7'

In [ ]:
# !pip install ray[tune]
!pip install ax-platform==0.2.4

In [ ]:
import numpy as np
import time

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch

Let's start by defining a classic benchmark for global optimization.
The form here is explicit for demonstration, yet it is typically a black-box.
We artificially sleep for a bit (`0.02` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune 6-dimensions of the `x` hyperparameter.

In [ ]:
def landscape(x):
    """
    Hartmann 6D function containing 6 local minima.
    It is a classic benchmark for developing global optimization algorithms.
    """
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array(
        [
            [10, 3, 17, 3.5, 1.7, 8],
            [0.05, 10, 17, 0.1, 8, 14],
            [3, 3.5, 1.7, 10, 17, 8],
            [17, 8, 0.05, 10, 0.1, 14],
        ]
    )
    P = 10 ** (-4) * np.array(
        [
            [1312, 1696, 5569, 124, 8283, 5886],
            [2329, 4135, 8307, 3736, 1004, 9991],
            [2348, 1451, 3522, 2883, 3047, 6650],
            [4047, 8828, 8732, 5743, 1091, 381],
        ]
    )
    y = 0.0
    for j, alpha_j in enumerate(alpha):
        t = 0
        for k in range(6):
            t += A[j, k] * ((x[k] - P[j, k]) ** 2)
        y -= alpha_j * np.exp(-t)
    return y

Next, our ``objective`` function takes a Tune ``config``, evaluates the `landscape` of our experiment in a training loop,
and uses `tune.report` to report the `landscape` back to Tune.

In [ ]:
def objective(config):
    for i in range(config["iterations"]):
        x = np.array([config.get("x{}".format(i + 1)) for i in range(6)])
        tune.report(
            timesteps_total=i, landscape=landscape(x), l2norm=np.sqrt((x ** 2).sum())
        )
        time.sleep(0.02)

In [ ]:
ray.init(configure_logging=False)

Now we define the search algorithm from `AxSearch`  with optional parameter and outcome constraints.

In [ ]:
algo = AxSearch(
    parameter_constraints=["x1 + x2 <= 2.0"],
    outcome_constraints=["l2norm <= 1.25"],
)

We also use `ConcurrencyLimiter` to constrain to 4 concurrent trials. We include a scheduler, `AsyncHyperBandScheduler`, to demonstrate the modularity of Ray Tune. 

In [ ]:
algo = tune.suggest.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()
num_samples = 100
stop_timesteps = 200

In [ ]:
# Reducing samples for smoke tests
num_samples = 10

Lastly, we run the experiment to find the global minimum of the provided landscape (which contains 5 false minima). The argument to metric, `"landscape"`, is provided via the `objective` function's `tune.report`.

In [ ]:
analysis = tune.run(
    objective,
    name="ax",
    metric="landscape",
    mode="min",
    search_alg=algo,
    scheduler=scheduler,
    num_samples=num_samples,
    config={
        "iterations":100,
        "x1": tune.uniform(0.0, 1.0),
        "x2": tune.uniform(0.0, 1.0),
        "x3": tune.uniform(0.0, 1.0),
        "x4": tune.uniform(0.0, 1.0),
        "x5": tune.uniform(0.0, 1.0),
        "x6": tune.uniform(0.0, 1.0)
    },
    stop={"timesteps_total": stop_timesteps}
)

Here are the results:

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
ray.shutdown()